#### Introduction to Statistical Learning, Lab 3.3

# Interactions & Non-linear Transformations

We often want to include interaction terms and non-linear transformations of the predictors in our model. This is fully supported by the formula mini language.


  - [statsmodels documentation](https://www.statsmodels.org/stable/)
  - [statsmodels formula interface](https://www.statsmodels.org/stable/example_formulas.html)
  - [the formula mini language](https://patsy.readthedocs.io/en/latest/formulas.html#the-formula-language)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from islpy import datasets
sns.set()
%matplotlib inline

#### Data Set

We use the `Boston` data set to demonstrate multiple linear regression.

In [ ]:
boston = datasets.Boston()
boston.head()

#### Model Specification

The `smf.ols()` function builds a statistical *model* prepared for fitting with *ordinary least squares* (ols). This is the type of fit explained in detail in the lecture.

The syntax to use interaction terms is `y~x1:x2`. This will include a term corresponding to $x_1\times x_2$ in the model.

There is a shorthand notation for including an interaction term and the predictors themselves: `y~x1*x2`. This is equivalent to `y~x1+x2+x1:x2`.

As in the simple regression with one predictor, a constant term for the intercept is added automatically.

The formula `medv~lstat*age` means we are using `lstat`, `age` and the interaction term `lstat`$\times$`age` as our predictors and `medv` as our dependent variable:

$$ \mathrm{medv} = \beta_0 + \beta_1 \mathrm{lstat} + \beta_2 \mathrm{age} + \beta_3 \mathrm{lstat}\times\mathrm{age}$$

In [ ]:
model = smf.ols(formula='medv~lstat*age', data=boston)

#### Fitting the Model

We *fit* the model to the data by calling the `fit()` method:

In [ ]:
model_fit = model.fit()

#### Fit Result Summary

We can get a comprehensive summary using the `summary()` method. Now we get the results for all three $\beta$ coefficients.

In [ ]:
model_fit.summary()

#### A Fancy 3D Plot

Oh well...

In [ ]:
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
sns.reset_defaults()
%matplotlib inline

fig = plt.figure(figsize=(12, 9))
ax = axes3d.Axes3D(fig)

# 3D scatter plot of the raw data
ax.scatter(boston.lstat, boston.age, boston.medv)

# prepare point grids from the ranges of the scatter plot
xs = np.linspace(*ax.get_xlim(), 100)
ys = np.linspace(*ax.get_ylim(), 100)
xv, yv = np.meshgrid(xs, ys, copy=False)
zv = np.zeros((ys.size, xs.size))
lv = np.zeros((ys.size, xs.size))
uv = np.zeros((ys.size, xs.size))

# compute predictions and CI bounds for the rows in the point grids
for idx, y in enumerate(yv):
    pred = model_fit.get_prediction({'lstat': xs, 'age': y}).summary_frame()
    zv[idx] = pred['mean']
    lv[idx] = pred['mean_ci_lower']
    uv[idx] = pred['mean_ci_upper']

# plot the prediction & CI boundary surfaces
ax.plot_surface(xv, yv, zv, alpha=0.4)
ax.plot_surface(xv, yv, lv, alpha=0.2, color='C1')
ax.plot_surface(xv, yv, uv, alpha=0.2, color='C1')

# add contour plot of the CI width to the bottom of the figure
ax.contourf(xv, yv, uv-lv,
            zdir='z',
            offset=ax.get_zlim()[0],
            levels=30,
            antialiased=True,
            cmap='Oranges')

# set figure title and axes labels
ax.set_title('Linear regression on Boston housing data: medv ~ lstat * age')
ax.set_xlabel('lstat')
ax.set_ylabel('age')
ax.set_zlabel('medv')

# specify viewing angle
# ax.view_init(15, -70)